In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('rating3.csv',usecols=[0,1,2])

In [3]:
df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [4]:
df = df.sample(500000)

In [5]:
raters=df.groupby(['movieId'])['rating'].count()

In [6]:
data=df.merge(raters,on='movieId')
data['count']=data['rating_y']
data['rating']= data['rating_x']
data.drop(['rating_x','rating_y'],axis=1,inplace=True)

In [7]:
data.head()

,userId,movieId,count,rating
0,1151,318,1529,4.0
1,6618,318,1529,5.0
2,409,318,1529,5.0
3,5558,318,1529,2.0
4,2187,318,1529,5.0


In [8]:
data.shape

(500000, 4)

In [9]:
selfjoined = data.merge(data,on='userId')

In [10]:
selfjoined.head()

,userId,movieId_x,count_x,rating_x,movieId_y,count_y,rating_y
0,1151,318,1529,4.0,318,1529,4.0
1,1151,318,1529,4.0,434,648,3.0
2,1151,318,1529,4.0,10,715,5.0
3,1151,318,1529,4.0,380,1094,5.0
4,1151,318,1529,4.0,588,1036,4.0


In [11]:
selfjoined = selfjoined[selfjoined['movieId_x']<selfjoined['movieId_y']]

In [12]:
import numpy as np
selfjoined['r1r2'] = selfjoined['rating_x']*selfjoined['rating_y']
selfjoined['r1square'] = np.square(selfjoined['rating_x'])
selfjoined['r2square'] = np.square(selfjoined['rating_y'])

In [13]:
selfjoined.head()

,userId,movieId_x,count_x,rating_x,movieId_y,count_y,rating_y,r1r2,r1square,r2square
1,1151,318,1529,4.0,434,648,3.0,12.0,16.0,9.0
3,1151,318,1529,4.0,380,1094,5.0,20.0,16.0,25.0
4,1151,318,1529,4.0,588,1036,4.0,16.0,16.0,16.0
5,1151,318,1529,4.0,593,1547,5.0,20.0,16.0,25.0
7,1151,318,1529,4.0,344,948,4.0,16.0,16.0,16.0


In [14]:
aggdata=selfjoined.groupby(['movieId_x','movieId_y'])['rating_x','rating_y','r1r2','r1square','r2square','count_x','count_y'].sum()
aggdata.head()

rating_x  rating_y     r1r2  r1square  r2square  count_x  \
movieId_x movieId_y                                                             
1         2             767.5     608.5  2482.75   3190.25   2085.25   241042   
          3             421.5     339.5  1360.00   1757.75   1201.75   133772   
          4             105.5      76.5   305.00    435.25    233.75    34074   
          5             462.5     368.0  1502.75   1929.75   1272.50   147654   
          6             731.5     715.0  2906.25   3057.75   2952.50   228422   

                     count_y  
movieId_x movieId_y           
1         2           106769  
          3            34026  
          4             1863  
          5            37791  
          6           101722

In [15]:
n=aggdata.shape[0]

In [16]:
aggdata['corelation'] = (n*aggdata['r1r2'] - aggdata['rating_x']*aggdata['rating_y']) / \
np.sqrt((n * aggdata['r1square']  - np.square(aggdata['rating_x']))*(n * aggdata['r2square']  - np.square(aggdata['rating_y'])))

In [17]:
aggdata['corelation'] = aggdata['corelation']*n/(n+50)

In [18]:
aggdata.reset_index(inplace=True)
aggdata.head()

,movieId_x,movieId_y,rating_x,rating_y,r1r2,r1square,r2square,count_x,count_y,corelation
0,1,2,767.5,608.5,2482.75,3190.25,2085.25,241042,106769,0.962587
1,1,3,421.5,339.5,1360.00,1757.75,1201.75,133772,34026,0.935733
2,1,4,105.5,76.5,305.00,435.25,233.75,34074,1863,0.956210
3,1,5,462.5,368.0,1502.75,1929.75,1272.50,147654,37791,0.958972
4,1,6,731.5,715.0,2906.25,3057.75,2952.50,228422,101722,0.967243


In [19]:
movies = pd.read_csv('movies.csv',usecols=[0,1])

In [20]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [21]:
aggdata1=aggdata.merge(movies,left_on='movieId_x',right_on='movieId')
aggdata2=aggdata1.merge(movies,left_on='movieId_y',right_on='movieId')

In [22]:
aggdata2.head()

,movieId_x,movieId_y,rating_x,rating_y,r1r2,r1square,r2square,count_x,count_y,corelation,movieId_x,title_x,movieId_y,title_y
0,1,2,767.5,608.5,2482.75,3190.25,2085.25,241042,106769,0.962587,1,Toy Story (1995),2,Jumanji (1995)
1,1,3,421.5,339.5,1360.00,1757.75,1201.75,133772,34026,0.935733,1,Toy Story (1995),3,Grumpier Old Men (1995)
2,2,3,200.5,193.0,678.00,740.75,681.00,32981,18939,0.954594,2,Jumanji (1995),3,Grumpier Old Men (1995)
3,1,4,105.5,76.5,305.00,435.25,233.75,34074,1863,0.956210,1,Toy Story (1995),4,Waiting to Exhale (1995)
4,2,4,63.0,57.0,200.00,243.00,193.00,10062,1242,0.923521,2,Jumanji (1995),4,Waiting to Exhale (1995)


In [23]:
def recommendation(title):
    recommended_movies = []
    data =aggdata2[aggdata2['title_x']==title]
    data = data.sort_values(by='corelation',ascending=False)
    return data

In [24]:
collabaritive=recommendation('Toy Story (1995)')[['title_y','corelation']]
collabaritive['title'] = collabaritive['title_y'].str.split(' \(').str.get(0)
collabaritive.drop('title_y',axis=1,inplace=True)
collabaritive.head()

,corelation,title
15344850,0.999997,Backcountry
11469398,0.999997,"Man Who Never Was, The"
4000855,0.999997,Tuvalu
4015885,0.999997,Satyricon
4018449,0.999997,Frankie and Johnny


In [25]:
contentrating = pd.read_csv('contentrating.csv')

In [26]:
contentrating.head()

,Unnamed: 0,0,1
0,0,Toy Story,1.000000
1,0,Toy Story 3,0.228218
2,0,Song of the Sea,0.175412
3,0,Inside Out,0.166091
4,0,Finding Nemo,0.146385


In [27]:
hybrid=contentrating.merge(collabaritive,left_on='0',right_on='title')
hybrid=hybrid[['title','1','corelation']]
hybrid['wcorelation'] = (hybrid['1'] + hybrid['corelation'])/2

In [28]:
hybrid.head()

,title,1,corelation,wcorelation
0,Toy Story 3,0.228218,0.987380,0.607799
1,Song of the Sea,0.175412,0.999997,0.587704
2,Finding Nemo,0.146385,0.980162,0.563273
3,Aladdin,0.143019,0.974068,0.558543
4,Up,0.136931,0.974051,0.555491


In [29]:
hybrid.sort_values(by='wcorelation',ascending=False).head(10)[['title','wcorelation']]

,title,wcorelation
0,Toy Story 3,0.607799
1,Song of the Sea,0.587704
2,Finding Nemo,0.563273
5,"Monsters, Inc.",0.559965
3,Aladdin,0.558543
4,Up,0.555491
6,Mary and Max,0.546135
7,Stand by Me,0.538058
12,WALL·E,0.536984
13,How to Train Your Dragon,0.534608
